<a href="https://colab.research.google.com/github/Astroneko404/TF_Study_Note/blob/master/CIFAR-10%20Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf

In [6]:
tf.config.experimental_run_functions_eagerly(True)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

170500096/170498071 [==============================] - 4s 0us/step


In [7]:
# Flattening
x_train_flattened = x_train.reshape(x_train.shape[0], 32*32*3)
x_test_flattened = x_test.reshape(x_test.shape[0], 32*32*3)

# Gray-scale
x_train_gray = tf.image.rgb_to_grayscale(x_train)
x_test_gray = tf.image.rgb_to_grayscale(x_test)

print("x_train:", x_train.shape)
print("x_test:", x_test.shape)
print("x_train_flattened:", x_train_flattened.shape)
print("x_train_grayscale:", x_train_gray.shape)

x_train: (50000, 32, 32, 3)
x_test: (10000, 32, 32, 3)
x_train_flattened: (50000, 3072)
x_train_grayscale: (50000, 32, 32, 1)


In [8]:
print("y_train", y_train.shape) # Labels ranging from 0 to 9

y_train (50000, 1)


### 1. Using LSTM
Image sets are preprocessed using *tf.image.rgb_to_grayscale()*

In [10]:
from tensorflow.keras import models, layers, optimizers

inputs = layers.Input(shape=(32, 32), name='inputs')
lstm_layer = layers.LSTM(units=256, return_sequences=False)(inputs)
outputs = layers.Dense(10, activation='softmax')(lstm_layer)
lstm = tf.keras.Model(inputs, outputs)

lstm.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
lstm.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 32, 32)]          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               295936    
_________________________________________________________________
dense (Dense)                (None, 10)                2570      
Total params: 298,506
Trainable params: 298,506
Non-trainable params: 0
_________________________________________________________________


In [12]:
lstm.fit(x_train_gray, y_train, batch_size=32, epochs=5, validation_split=0.1)
lstm.evaluate(x_test_gray,  y_test, verbose=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/5
1407/1407 [==============================] - 254s 180ms/step - loss: 1.9118 - accuracy: 0.3000 - val_loss: 1.7485 - val_accuracy: 0.3728
Epoch 2/5
1407/1407 [==============================] - 241s 171ms/step - loss: 1.6902 - accuracy: 0.3913 - val_loss: 1.6096 - val_accuracy: 0.4222
Epoch 3/5
1407/1407 [==============================] - 240s 171ms/step - loss: 1.5638 - accuracy: 0.4393 - val_loss: 1.5362 - val_accuracy: 0.4408
Epoch 4/5
1407/1407 [==============================] - 257s 183ms/step - loss: 1.4709 - accuracy: 0.4729 - val_loss: 1.4554 - val_accuracy: 0.4828
Epoch 5/5
1407/1407 [==============================] - 253s 180ms/step - loss: 1.3908 - accuracy: 0.5025 - val_loss: 1.3996 - val_accuracy: 0.4972
313/313 - 21s - loss: 1.4280 - accuracy: 0.4931


[1.4280476570129395, 0.49309998750686646]

### 2. Using CNN

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)               

In [20]:
model.fit(x_train, y_train, batch_size=32, epochs=8, validation_split=0.1)
model.evaluate(x_test, y_test, verbose=2)

Epoch 1/8
   2/1407 [..............................] - ETA: 44s - loss: 2.0759 - accuracy: 0.1875

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


1407/1407 [==============================] - 107s 76ms/step - loss: 1.5025 - accuracy: 0.4532 - val_loss: 1.2808 - val_accuracy: 0.5530
Epoch 2/8
1407/1407 [==============================] - 106s 75ms/step - loss: 1.1805 - accuracy: 0.5828 - val_loss: 1.0987 - val_accuracy: 0.6172
Epoch 3/8
1407/1407 [==============================] - 107s 76ms/step - loss: 1.0391 - accuracy: 0.6326 - val_loss: 0.9880 - val_accuracy: 0.6524
Epoch 4/8
1407/1407 [==============================] - 107s 76ms/step - loss: 0.9368 - accuracy: 0.6690 - val_loss: 0.9461 - val_accuracy: 0.6742
Epoch 5/8
1407/1407 [==============================] - 105s 75ms/step - loss: 0.8615 - accuracy: 0.6968 - val_loss: 0.9095 - val_accuracy: 0.6842
Epoch 6/8
1407/1407 [==============================] - 106s 75ms/step - loss: 0.8034 - accuracy: 0.7189 - val_loss: 0.8301 - val_accuracy: 0.7094
Epoch 7/8
1407/1407 [==============================] - 105s 75ms/step - loss: 0.7441 - accuracy: 0.7379 - val_loss: 0.8883 - val_accur

[0.8997931480407715, 0.6969000101089478]